In [2]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch
device = torch.device('cpu')

model = VisionEncoderDecoderModel.from_pretrained("checkpoints/checkpoint-90000")
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

image = Image.open("sample_imgs/1.png").convert("RGB")

pixel_values = (processor(image, return_tensors="pt").pixel_values).to(device)
generated_ids = model.generate(pixel_values)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] 
generated_ids, generated_text

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


(tensor([[ 101,  101, 3136,  679, 1962, 8024,  679, 3221, 2769,  812, 1962, 2345,
           818, 8038,  791, 1921, 2218, 6382, 1168, 6821, 7027,  711,  102]]),
 '教 不 好 ， 不 是 我 们 好 差 任 ： 今 天 就 讲 到 这 里 为')